In [11]:
import os

from langchain_community.document_loaders import PyPDFLoader

In [6]:
DATA_PATH = r'data\textbook.pdf'

loader = PyPDFLoader(DATA_PATH)
document = loader.load()

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 100,
    add_start_index = True
)

chunks = text_splitter.split_documents(document)

len(document), len(chunks)

(1232, 15069)

In [10]:
document = chunks[42]

print(document.page_content)
print(document.metadata)

noise level. Finally, test the algorithm on real-world data, preferably drawn from a wide
variety of sources, such as photos found on the web. Only then can you truly know if your
algorithm can deal with real-world complexity, i.e., images that do not ﬁt some simpliﬁed
model or assumptions.
{'source': 'data\\textbook.pdf', 'page': 8, 'start_index': 353}
